<h1 style="font-size:3rem;color:orange;">Eleições 2022 - Análise do Tempo de Votação</h1>

AVISO de Licença de Uso e Reserva de Direitos Autorais BSD 3 Clause. Copyright (c) 2022 por John Robson <john.robson@msn.com> (PIX)

Todo o Código fonte e demais arquivos estão sobre a Licença BSD 3 Clause. Modificações, Redistribuição, uso Comercial são permitidos, sempre informando
esse aviso de direitos autorais e o repositório oficial: https://github.com/JohnRobson/Eleicoes2022

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import timeit
import ast
from sqlalchemy import select, func

import matplotlib.pylab as plt
%matplotlib inline

from IPython.display import Markdown, display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>div.output_scroll { height: 100%; }</style>"))

from bd_urnas import engine, Base, Session, Urnas
from src_config import federacao, estados_federacao
session = Session()

def printmd(string): # convert markdown para HTML
    display(Markdown(string))

In [3]:
# Importando Banco de Dados para Python Pandas Dataframe
bd = pd.read_sql_query('SELECT id, turno, regiao, estado, municipio, zona, local, secao, vPresA22, vPresB13, vPresNulo, vPresBranco,\
vGovA, vGovB, vGovNulo, vGovBranco, qEleitAptos, qEleitAptos, qComparecimento, qEleitLibCodigo, qEleitBiometrico, vPresTotal, modelUrna,\
vConfPres10min, arquivoBU FROM urnas', session.bind)
bd

OperationalError: (sqlite3.OperationalError) no such column: vConfPres10min
[SQL: SELECT id, turno, regiao, estado, municipio, zona, local, secao, vPresA22, vPresB13, vPresNulo, vPresBranco,vGovA, vGovB, vGovNulo, vGovBranco, qEleitAptos, qEleitAptos, qComparecimento, qEleitLibCodigo, qEleitBiometrico, vPresTotal, modelUrna,vConfPres10min, arquivoBU FROM urnas]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Calculando Votos Intervalo de Tempo Entre os Votos Registrados

In [ ]:
alto_comparecimento = bd.sort_values(by=['qComparecimento'], ascending=False) # ordena por quantidade de comparecimento
cols = ['vPresIntervalo', 'vConfPres10min'] # se não existir registro de votos remove a urna 
alto_comparecimento[cols] = alto_comparecimento[cols].replace({'0':np.nan, 0:np.nan})
alto_comparecimento.dropna(subset=alto_comparecimento.loc[:0, cols].columns, inplace=True)

m1 = 0; m2 = 0

for i in range(2):
    urna = alto_comparecimento.iloc[i]
    printmd(f'{i} - {urna["regiao"]}, {urna["estado"]}, {urna["municipio"]}, {urna["zona"]}, {urna["secao"]}, {urna["local"]} - Comparecimento: {urna["qComparecimento"]}')

    fig, ax = plt.subplots(1, 3, gridspec_kw={'width_ratios': [10, 1, 2]}, figsize=(12, 5))
    
    votos10min = dict(ast.literal_eval(urna['vConfPres10min']).items()) # lê o string_dict e converte para dict
    x, y = zip(*votos10min.items())
    x = [str(i) for i in x] # converter x de int para str
    ax[0].bar(x, y);
    ax[0].title.set_text('Quantidade de Votos a Cada 10 Minutos')
    ax[0].tick_params(axis='x', labelrotation=90, labelsize=8)
    ax[0].set_ylim([0, 25])
    ax[0].margins(0.01, None)
    m1 = max(y) if max(y) >= m1 else m1
    
    menores_intervalos_voto = list(map(int, urna['vPresIntervalo'].split(','))) # converte string de int para lista de int
    ax[1].boxplot(menores_intervalos_voto, notch=True, patch_artist=True, boxprops=dict(facecolor='lightblue'));
    ax[1].title.set_text('Votos Rápidos')
    ax[1].get_xaxis().set_ticks([])
    
    votos60min = pd.DataFrame(votos10min.values(), index=votos10min.keys()) # converte o dicionário para um dataframe
    votos60min = votos60min.groupby(votos60min.index // 6).sum().to_dict()[0] # soma a cada 6 linhas e converte para dict
    x, y = zip(*votos60min.items())
    x = [str(i) for i in x] # converter x de int para str
    ax[2].bar(x, y);
    ax[2].title.set_text('Qtd de Votos Por Hora')
    ax[2].tick_params(axis='x', labelsize=8)
    ax[2].set_ylim([0, 125])
    m2 = max(y) if max(y) >= m2 else m2
    
    fig.tight_layout();
    plt.show();

printmd(f'Máximo a cada 10 min {m1} e 1 hora {m2}. Analisados {i} urnas em ordem de quantidade de comparecimento.')

In [ ]:
data = [] # lista de dicionários
for index, row in alto_comparecimento.iterrows():
    data.append(ast.literal_eval(row['vConfPres10min'])) # converte os items da columa em dicionários e adiciona na lista

votosPres10min = pd.DataFrame(data, index=alto_comparecimento.index, columns=list(range(0, 61)))
votosPres10min

## REVISAR ABAIXO

In [ ]:
votosPres10min.iloc[:, 0].max()
votosPres10min.max()
votosPres10min_max = votosPres10min.max().max()
printmd(f'Máximo e Votos a cada 1 Hora: {votosPres10min_max}, ou 1 voto a cada {round((60 * 60) / votosPres10min_max, 1)} segundos.')
votosPres10min_mean = votosPres10min.mean().mean()
printmd(f'Média de Votos a cada 1 Hora: {round(votosPres10min_mean, 1)}, ou 1 voto a cada {round((60 * 60) / votosPres10min_mean, 1)} segundos.')